In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [3]:
quote_file = "../data/quotes_mp_sample/2020-07-28-mt.csv"
trade_file = "../data/trades_mp_sample/2020-07-28-mt.csv"

In [4]:
import sys
import pandas
import numpy
import requests
import os
import pandas as pd 
import numpy as np 
import datetime
from datetime import timezone
import utils


# Sample Time Stamp Cutoffs
t_beg = datetime.datetime.fromisoformat('2017-01-06 03:59:00.000000-05:00')
t1 = datetime.datetime.fromisoformat('2017-01-06 04:00:00.000000-05:00')
t2 = datetime.datetime.fromisoformat('2017-01-06 04:00:01.000000-05:00')
t3 = datetime.datetime.fromisoformat('2017-01-06 04:00:02.000000-05:00')
t4 = datetime.datetime.fromisoformat('2017-01-06 04:00:06.000000-05:00')
t5 = datetime.datetime.fromisoformat('2017-01-06 04:01:06.000000-05:00')
t6 = datetime.datetime.fromisoformat('2017-01-06 04:10:06.000000-05:00')
t_end = datetime.datetime.fromisoformat('2019-01-06 22:31:00.000000-05:00')
t_indices = [t_beg, t1, t2, t3, t4, t5, t6, t_end] 

keys = ['Ask Price', 'Ask Size', 'Bid Price', 'Bid Size']

df_quotes = pd.read_csv(quote_file)
df_trades = pd.read_csv(trade_file)

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (msal 1.0.0 (c:\users\amah\.conda\envs\amlcomponentsdk\lib\site-packages), Requirement.parse('msal<2.0.0,>=1.3.0'), {'azure-identity'}).


In [5]:
df_quotes['Date-Time'] = pd.to_datetime(df_quotes['Date-Time'])
df_trades['Date-Time'] = pd.to_datetime(df_trades['Date-Time'])
df_quotes = utils.trim_df_to_time(df_quotes, "8:30:00.00", "15:00:00.00")
df_trades = utils.trim_df_to_time(df_trades, "8:30:00.00", "15:00:00.00")
# Clean Quotes and Trades
cleaner = utils.data_process()

full df shape (1222172, 10)
>8:30  df shape (1014976, 10)
time-trimmed df shape (877168, 10)
full df shape (181517, 7)
>8:30  df shape (148736, 7)
time-trimmed df shape (126925, 7)


In [6]:
# Quotes
cleaner.set_negatives_to_nan(df_quotes, keys=['Ask Price', 'Bid Price', 'Ask Size', 'Bid Size'])
cleaner.drop_nans(df_quotes, keys=['Ask Price', 'Bid Price', 'Ask Size', 'Bid Size'])

# Trades
# Drop zero/negative 
cleaner.set_negatives_to_nan(df_trades, keys=['Price', 'Acc Volume'])
cleaner.drop_nans(df_trades, keys=['Price', 'Acc Volume'])

# Detect outliers
cleaner.detect_outliers(df_trades, 40, 6, 'Price')
df_trades['Price'] = np.where(df_trades['Outlier'], np.nan, df_trades['Price'])
df_trades['Acc Volume'] = np.where(df_trades['Outlier'], np.nan, df_trades['Acc Volume'])
cleaner.set_negatives_to_nan(df_trades, keys=['Price', 'Acc Volume'])
cleaner.drop_nans(df_trades, keys=['Price', 'Acc Volume'])

# Forward fill quotes
for key in keys:
    df_quotes[key] = df_quotes[key].fillna(method='ffill')

In [7]:

# Form lag(price) with distinct value
df_trades['temp_lag_price'] = df_trades['Price'].shift(1)
df_trades['Lag(Price) Distinct'] = np.where(df_trades['temp_lag_price'] == df_trades['Price'], np.NaN,
                                            df_trades['temp_lag_price'])
df_trades['Lag(Price) Distinct'].ffill(inplace=True)
df_trades.drop('temp_lag_price', 1, inplace=True)

# Merge Quotes and Trades
df_quotes['Type'] = 'Quote'
df_all = pd.concat([df_quotes[['Date-Time', 'Type', 'Ask Price', 'Bid Price', 'Seq. No.']],
        df_trades[['Date-Time', 'Price', 'Lag(Price) Distinct', 'Acc Volume', 'Seq. No.']]], sort=True).sort_values(by=['Date-Time', 'Seq. No.'])



In [24]:
df_trades_all.head(100)

,Acc Volume,Ask Price,Bid Price,Date-Time,Lag(Price) Distinct,Price,Seq. No.,Type,Mid Quote,case1,case2,case3,case4,Tick Dir,Signed Trade SQRT,Signed Trade
0,159129.0,3223.75,3223.50,2020-07-28 08:30:00.008937042-05:00,NaN,3223.500,9955222.0,Trade,3223.625,0,-1,0,0,-1,-398.909764,-159129.0
1,159131.0,3223.75,3223.50,2020-07-28 08:30:00.012660080-05:00,3223.500,3223.625,9955225.0,Trade,3223.625,0,0,1,0,1,398.912271,159131.0
2,159138.0,3223.50,3223.50,2020-07-28 08:30:00.016637045-05:00,3223.625,3223.500,9955256.0,Trade,3223.500,0,0,0,-1,-1,-398.921045,-159138.0
3,159141.0,3223.50,3223.50,2020-07-28 08:30:00.016743680-05:00,3223.625,3223.500,9955266.0,Trade,3223.500,0,0,0,-1,-1,-398.924805,-159141.0
4,159143.0,3223.50,3223.25,2020-07-28 08:30:00.016807096-05:00,3223.625,3223.500,9955273.0,Trade,3223.375,1,0,0,0,1,398.927312,159143.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,159831.0,3223.25,3223.00,2020-07-28 08:30:00.420003100-05:00,3223.000,3223.250,9956213.0,Trade,3223.125,1,0,0,0,1,399.788694,159831.0
96,159832.0,3223.25,3223.00,2020-07-28 08:30:00.420573326-05:00,3223.000,3223.250,9956220.0,Trade,3223.125,1,0,0,0,1,399.789945,159832.0
97,159835.0,3223.25,3223.25,2020-07-28 08:30:00.440855148-05:00,3223.000,3223.250,9956231.0,Trade,3223.250,0,0,1,0,1,399.793697,159835.0
98,159838.0,3223.25,3223.25,2020-07-28 08:30:00.440919956-05:00,3223.000,3223.250,9956238.0,Trade,3223.250,0,0,1,0,1,399.797449,159838.0


In [9]:
df_all['Type'].fillna(value='Trade', inplace=True)
df_all['Ask Price'].ffill(inplace=True)
df_all['Bid Price'].ffill(inplace=True)
df_trades_all = df_all[df_all['Type'] == 'Trade'].copy(deep=True)

In [10]:
df_trades_all.reset_index(drop=True,inplace=True)
df_trades_all.head(100)

,Acc Volume,Ask Price,Bid Price,Date-Time,Lag(Price) Distinct,Price,Seq. No.,Type
0,159129.0,3223.75,3223.50,2020-07-28 08:30:00.008937042-05:00,NaN,3223.500,9955222.0,Trade
1,159131.0,3223.75,3223.50,2020-07-28 08:30:00.012660080-05:00,3223.500,3223.625,9955225.0,Trade
2,159138.0,3223.50,3223.50,2020-07-28 08:30:00.016637045-05:00,3223.625,3223.500,9955256.0,Trade
3,159141.0,3223.50,3223.50,2020-07-28 08:30:00.016743680-05:00,3223.625,3223.500,9955266.0,Trade
4,159143.0,3223.50,3223.25,2020-07-28 08:30:00.016807096-05:00,3223.625,3223.500,9955273.0,Trade
...,...,...,...,...,...,...,...,...
95,159831.0,3223.25,3223.00,2020-07-28 08:30:00.420003100-05:00,3223.000,3223.250,9956213.0,Trade
96,159832.0,3223.25,3223.00,2020-07-28 08:30:00.420573326-05:00,3223.000,3223.250,9956220.0,Trade
97,159835.0,3223.25,3223.25,2020-07-28 08:30:00.440855148-05:00,3223.000,3223.250,9956231.0,Trade
98,159838.0,3223.25,3223.25,2020-07-28 08:30:00.440919956-05:00,3223.000,3223.250,9956238.0,Trade


In [11]:
df_trades_all.shape

(126662, 8)

In [12]:
# Mid Quote
df_trades_all['Mid Quote'] = (df_trades_all['Ask Price'] + df_trades_all['Bid Price']) / 2

# Finding Tick Dir (Lee and Ready)
# Buy
df_trades_all['case1'] = np.where(df_trades_all['Price'] > df_trades_all['Mid Quote'], 1, 0)
# Sell
df_trades_all['case2'] = np.where(df_trades_all['Price'] < df_trades_all['Mid Quote'], -1, 0)
# Buy
df_trades_all['case3'] = np.where((df_trades_all['Price'] == df_trades_all['Mid Quote']) & (df_trades_all['Price'] > df_trades_all['Lag(Price) Distinct']), 1, 0)
# Sell
df_trades_all['case4'] = np.where((df_trades_all['Price'] == df_trades_all['Mid Quote']) & (df_trades_all['Price'] < df_trades_all['Lag(Price) Distinct']), -1, 0)

df_trades_all['Tick Dir'] = df_trades_all['case1'] + df_trades_all['case2'] + df_trades_all['case3'] + df_trades_all['case4']
df_trades_all['Signed Trade SQRT'] = df_trades_all['Tick Dir'] * np.sqrt(df_trades_all['Acc Volume'])
df_trades_all['Signed Trade'] = df_trades_all['Tick Dir'] * (df_trades_all['Acc Volume'])

In [14]:
df_all.tail(30)

,Acc Volume,Ask Price,Bid Price,Date-Time,Lag(Price) Distinct,Price,Seq. No.,Type
877143,NaN,3211.75,3211.5,2020-07-28 14:59:59.924831714-05:00,NaN,NaN,15246751.0,Quote
877144,NaN,3211.75,3211.5,2020-07-28 14:59:59.928850348-05:00,NaN,NaN,15246760.0,Quote
877145,NaN,3211.75,3211.5,2020-07-28 14:59:59.936840770-05:00,NaN,NaN,15246768.0,Quote
877146,NaN,3211.75,3211.5,2020-07-28 14:59:59.940244422-05:00,NaN,NaN,15246778.0,Quote
877147,NaN,3211.75,3211.5,2020-07-28 14:59:59.944842221-05:00,NaN,NaN,15246786.0,Quote
877148,NaN,3211.75,3211.5,2020-07-28 14:59:59.948845677-05:00,NaN,NaN,15246788.0,Quote
877149,NaN,3211.75,3211.5,2020-07-28 14:59:59.952121210-05:00,NaN,NaN,15246792.0,Quote
877150,NaN,3211.75,3211.5,2020-07-28 14:59:59.956293704-05:00,NaN,NaN,15246801.0,Quote
877151,NaN,3211.75,3211.5,2020-07-28 14:59:59.956363416-05:00,NaN,NaN,15246818.0,Quote
126920,1227048.0,3211.75,3211.5,2020-07-28 14:59:59.956363416-05:00,3211.75,3211.50,15246831.0,Trade


In [15]:
# Calculate some instantanous measures
df_quotes['Spread'] = df_quotes['Ask Price'] - df_quotes['Bid Price']
df_quotes['Mid Quote'] = (df_quotes['Ask Price'] + df_quotes['Bid Price']) / 2
df_quotes['Smart Price'] = (df_quotes['Ask Price']*(1/df_quotes['Ask Size']) + df_quotes['Bid Price']*(1/df_quotes['Bid Size'])) / (1/df_quotes['Ask Size'] + 1/df_quotes['Bid Size'])
df_quotes['Quote Imbalance'] = np.log(df_quotes['Ask Size']) - np.log(df_quotes['Bid Size'])

keys = keys + ['Spread', 'Mid Quote', 'Smart Price', 'Quote Imbalance']

In [16]:
# df_TW_avg.head()

In [17]:
# calulating the tick indices
vol = df_trades_all["Acc Volume"]
vol_tick = 1000
vol_levels = range(int(min(vol)) + vol_tick, int(max(vol)), vol_tick)
tick_times = []
trades_inds=[]
for level in vol_levels:
    ind = vol[vol > level].index.min()
    trades_inds.append(ind)
    tick_times.append(df_trades_all.loc[ind, "Date-Time"])
print("The tick_times for ticks(5):\n {}".format(tick_times[:5]))
times = df_quotes["Date-Time"]
inds = []
for tick_time in tick_times:
    ind = times[times < tick_time].index.max()
    inds.append(ind)
inds = [x for x in inds if x > 0]  # remove nans
print("The indices for ticks (5):\n {}".format(inds[:5]))
inds = sorted(list(set(inds)))
inds_enclosed = [0] + inds
# inds_timestamps = df_quotes.loc[inds, "Date-Time"].values
# print("The timestmps for indices for ticks:\n {}".format(inds_timestamps[:5]))
df_quotes.dropna(inplace=True)
print("len inds:{}".format(len(inds)))
inds = sorted(list(set(inds)))
inds_enclosed = [0] + inds

The tick_times for ticks(5):
 [Timestamp('2020-07-28 08:30:00.932557041-0500', tz='pytz.FixedOffset(-300)'), Timestamp('2020-07-28 08:30:02.453102352-0500', tz='pytz.FixedOffset(-300)'), Timestamp('2020-07-28 08:30:03.044722469-0500', tz='pytz.FixedOffset(-300)'), Timestamp('2020-07-28 08:30:03.624819211-0500', tz='pytz.FixedOffset(-300)'), Timestamp('2020-07-28 08:30:04.713056661-0500', tz='pytz.FixedOffset(-300)')]
The indices for ticks (5):
 [260, 573, 723, 919, 1182]
len inds:1067


In [18]:
trades_times = df_trades_all["Date-Time"]
trades_inds = []
for ind in inds:
    trade_ind = trades_times[trades_times > df_quotes.loc[ind, "Date-Time"]].index.min()
    trades_inds.append(trade_ind)

In [36]:
print(len(trades_inds))
trades_inds[:5]

1067


[145, 247, 329, 435, 574]

In [21]:
df_quotes.loc[260, "Date-Time"]

Timestamp('2020-07-28 08:30:00.921637932-0500', tz='pytz.FixedOffset(-300)')

In [23]:
df_trades_all.loc[trades_inds]

,Acc Volume,Ask Price,Bid Price,Date-Time,Lag(Price) Distinct,Price,Seq. No.,Type,Mid Quote,case1,case2,case3,case4,Tick Dir,Signed Trade SQRT,Signed Trade
145,160126.0,3223.50,3223.25,2020-07-28 08:30:00.932484727-05:00,3223.250000,3223.500000,9957095.0,Trade,3223.375,1,0,0,0,1,400.157469,160126.0
247,161158.0,3224.00,3223.75,2020-07-28 08:30:02.453102352-05:00,3223.750000,3224.000000,9958868.0,Trade,3223.875,1,0,0,0,1,401.444890,161158.0
329,162175.0,3224.75,3224.50,2020-07-28 08:30:03.044722469-05:00,3224.750000,3225.000000,9959872.0,Trade,3224.625,1,0,0,0,1,402.709573,162175.0
435,163226.0,3226.50,3226.25,2020-07-28 08:30:03.624819211-05:00,3226.250000,3226.500000,9961432.0,Trade,3226.375,1,0,0,0,1,404.012376,163226.0
574,164133.0,3225.75,3225.75,2020-07-28 08:30:04.713056661-05:00,3226.000000,3225.750000,9963566.0,Trade,3225.750,0,0,0,-1,-1,-405.133311,-164133.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126370,1222202.0,3212.50,3212.25,2020-07-28 14:59:59.272044030-05:00,3212.250000,3212.500000,15242903.0,Trade,3212.375,1,0,0,0,1,1105.532451,1222202.0
126424,1223139.0,3212.50,3212.25,2020-07-28 14:59:59.360408292-05:00,3212.250000,3212.500000,15243517.0,Trade,3212.375,1,0,0,0,1,1105.956147,1223139.0
126474,1224149.0,3212.25,3212.00,2020-07-28 14:59:59.544280905-05:00,3212.250000,3212.057229,15244292.0,Trade,3212.125,0,-1,0,0,-1,-1106.412672,-1224149.0
126520,1225139.0,3212.50,3212.25,2020-07-28 14:59:59.624814859-05:00,3212.250000,3212.203704,15244929.0,Trade,3212.375,0,-1,0,0,-1,-1106.859973,-1225139.0


In [19]:
trades_inds=[0]+trades_inds

In [20]:
def safe_division(n, d):
    return n / d if d else 0

for i in range(len(trades_inds)-1):
    df_trades_all.loc[trades_inds[i+1],'Order Imbalance']=df_trades_all.loc[trades_inds[i]+1:trades_inds[i+1]]["Signed Trade"].sum()
    df_trades_all.loc[trades_inds[i+1],'Close Price']=df_trades_all.loc[trades_inds[i+1],'Price']
    df_trades_all.loc[trades_inds[i+1],'Open Price']=df_trades_all.loc[trades_inds[i],'Price']
    df_trades_all.loc[trades_inds[i+1],'Avg Trade Size']=df_trades_all.loc[trades_inds[i]+1:trades_inds[i+1]]["Acc Volume"].mean()
    df_trades_all.loc[trades_inds[i+1],'Net SellBuy Count']=safe_division(sum(df_trades_all.loc[trades_inds[i]+1:trades_inds[i+1]]["Tick Dir"]==-1),sum(df_trades_all.loc[trades_inds[i]+1:trades_inds[i+1]]["Tick Dir"]==1))
# 'Close Price', 'Avg Trade Size', 'Net Sell/Buy Count'=Sum Tick Dir, 'Ratio Sell/Buy=Sum(Tick Dir == -1)/sum(Tick Dir == 1)

In [31]:
# trades_inds.remove(0)
df_trades_ticks = df_trades_all.loc[trades_inds][["Order Imbalance", "Close Price", "Avg Trade Size", "Net SellBuy Count"]]
print("df_trades_ticks.shape", df_trades_ticks.shape)
print("df_trades_ticks columns", df_trades_ticks.columns)
keys = keys + ["Spread","Mid Quote","Smart Price","Quote Imbalance",]

df_trades_ticks.shape (1067, 4)
df_trades_ticks columns Index(['Order Imbalance', 'Close Price', 'Avg Trade Size',
       'Net SellBuy Count'],
      dtype='object')


In [32]:
df_trades_ticks.reset_index(inplace=True, drop=True)
df_trades_ticks.head()

,Order Imbalance,Close Price,Avg Trade Size,Net SellBuy Count
0,5278941.0,3223.50,159634.213793,0.629213
1,3211197.0,3224.00,160526.696078,0.672131
2,9378285.0,3225.00,161672.134146,0.171429
3,2604990.0,3226.50,162670.754717,0.737705
4,2441690.0,3225.75,163684.446043,0.805195


In [33]:
df_trades_ticks.shape

(1067, 4)

In [34]:
# Time weighted measures based on quotes
df_TW_avg = utils.TW_avg(
    input_df=df_quotes,
    datetime_col="Date-Time",
    keys=keys,
    timestamp_cutoffs=df_quotes.loc[
        df_quotes.index[inds_enclosed], "Date-Time"
    ].values,
    fillforward=True,
)

# Measures based on trades (Ali probably has the code and will need to match up with that)
# Ali to write sum 'Signed Trade' between two bars to get 'Order Imbalance'

# 'Close Price', 'Avg Trade Size', 'Net Sell/Buy Count'=Sum Tick Dir, 'Ratio Sell/Buy=Sum(Tick Dir == -1)/sum(Tick Dir == 1)

In [52]:
TW_avg_keys = ["TW Avg " + key for key in keys]
df_TW_avg.shape

(1324, 23)

In [42]:
df_bars = utils.set_df_labels(
    df_quotes,
    pt_level=float(0.005),
    sl_level=float(0.005),
    wait_time=timedelta(seconds=int(1000)),
    inds=inds,
)

In [51]:
df_bars.shape

(1324, 23)

In [53]:
df_bars = pd.concat([df_bars, df_TW_avg[TW_avg_keys], df_trades_ticks], axis=1)

In [55]:
df_bars.shape

(1324, 111)

In [15]:
date = '2017-06-01'
merged_df = pd.read_csv("{}/{}-mt.csv".format(path_to_merged, date))
merged_df.reset_index(level=0, inplace=True)
merged_df['Date-Time'] = pd.to_datetime(merged_df['Date-Time'])
# merged_df.head()

In [16]:
bars_df = pd.read_csv("{}/bars_{}-mt.csv".format(path_to_labels, date))
bars_df['Date-Time'] = pd.to_datetime(bars_df['Date-Time'])

In [17]:
start_time = '2017-06-01 9:00:01.149905303-05:00'
end_time = '2017-06-01 11:25:35.578865996-05:00'

In [18]:
merged_df = merged_df[merged_df['Date-Time']>start_time]
merged_df = merged_df[merged_df['Date-Time']<end_time]
bars_df = bars_df[bars_df['Date-Time']>start_time]
bars_df = bars_df[bars_df['Date-Time']<end_time]
print(merged_df.shape)
print(bars_df.shape)
inds = bars_df.index.to_list()

(181808, 10)
(436, 17)


In [19]:
def plot_barriers(i, df, merged_df, max_ind, pt_level=0.0001, sl_level=0.0001):
#     print(df.loc[i,'Bid Price']*np.array([1+pt_level,1-sl_level]), df.loc[i,'Date-Time'])
#     print(df.loc[i,'Price']*np.array([1+pt_level,1-sl_level]), df.loc[i,'Date-Time'], df.loc[merged_df.loc[i,'Date-Time']])
#     plt.hlines(df.loc[i,'Price']*np.array([1+pt_level,1-sl_level]), xmin=df.loc[i,'Date-Time'], xmax=df.loc[df.loc[i,'end_ind'],'Date-Time'], color='r', linestyles='--')
    c=['k','b','r'][df.loc[i,'long_label']]
#     print(df.loc[i,'label'], c)
    end_ind=min(int(df.loc[i,'end_ind']),max_ind)
    plt.hlines(df.loc[i,'Ask Price']*np.array([1+df.loc[i,'dailyVolatility']*pt_level,1-df.loc[i,'dailyVolatility']*sl_level]), xmin=df.loc[i,'Date-Time'], xmax=merged_df.loc[end_ind,'Date-Time'], color=c, linestyles='--')
    plt.vlines(x = [df.loc[i,'Date-Time'], merged_df.loc[end_ind,'Date-Time']], ymin=df.loc[i,'Ask Price']*(1-df.loc[i,'dailyVolatility']*sl_level), ymax=df.loc[i,'Ask Price']*(1+df.loc[i,'dailyVolatility']*pt_level), color=c, linestyles='--')
    plt.text(df.loc[i,'Date-Time'],df.loc[i,'Bid Price']+2,i )
# plot_barriers(i, bars_df)

In [20]:
# %matplotlib inline

In [21]:
%matplotlib widget

In [22]:
fig, ax = plt.subplots()

# fig.canvas.toolbar_visible = True
# fig.canvas.header_visible = False
# fig.canvas.resizable = True

merged_df.plot(x='Date-Time', y='Bid Price', style='.-', color='b', figsize=(12,6),ax=ax)
# merged_df.plot(x='Date-Time', y='Ask Price', style='.-', color='r', figsize=(12,6), ax=ax)
# merged_df.plot(y='Price', style='.', figsize=(16,12))
plt.xlabel("Time")
plt.ylabel("Price")
max_ind = merged_df.index.max()
for i in [inds[x] for x in range(0,len(inds),30)]:
#     print(i)
    plot_barriers(i, bars_df, merged_df,max_ind , pt_level=0.0003, sl_level=0.0003)
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
bars_df.loc[584,:]

original_index                                  277576
Date-Time          2017-06-01 10:46:50.142220892-05:00
Bid Price                                      2416.75
Ask Price                                         2417
Bid Size                                           596
Ask Size                                           288
Seq. No.                                       7440444
Exch Time                           15:46:50.133000000
#RIC                                              ESM7
dailyVolatility                                0.25142
pt_long_ind                                     278722
sl_long_ind                                        NaN
pt_short_ind                                       NaN
sl_short_ind                                    278723
end_ind                                         289383
long_label                                           1
short_label                                         -1
Name: 584, dtype: object

In [3]:
bars_path = "../data/test_label_mp_local/bars_2020-07-28-mt.csv"

In [4]:
bars_df = pd.read_csv(bars_path)

In [5]:
print(bars_df.shape)
bars_df.head()

(1067, 35)


,original_index,Date-Time,Bid Price,Ask Price,Bid Size,Ask Size,Seq. No.,Exch Time,#RIC,dailyVolatility,...,TW Avg Bid Price,TW Avg Bid Size,TW Avg Spread,TW Avg Mid Quote,TW Avg Smart Price,TW Avg Quote Imbalance,Order Imbalance,Close Price,Avg Trade Size,Net SellBuy Count
0,260,2020-07-28 08:30:00.921637932-05:00,3223.25,3223.50,300.0,163.0,9957088,13:30:00.914000000,ESU0,1.428027,...,3223.254389,192.124285,0.239882,3223.374330,3223.366940,0.030247,5278941.0,3223.50,159634.213793,0.629213
1,573,2020-07-28 08:30:02.452837685-05:00,3223.75,3224.00,360.0,1174.0,9958860,13:30:02.444000000,ESU0,1.428027,...,3223.440095,154.370824,0.256255,3223.568222,3223.533832,0.675698,3211197.0,3224.00,160526.696078,0.672131
2,723,2020-07-28 08:30:03.044659973-05:00,3224.50,3224.75,77.0,7.0,9959871,13:30:03.035000000,ESU0,1.428027,...,3224.307451,205.527718,0.249966,3224.432434,3224.456131,-0.641379,9378285.0,3225.00,161672.134146,0.171429
3,919,2020-07-28 08:30:03.622917398-05:00,3226.25,3226.50,312.0,35.0,9961425,13:30:03.583000000,ESU0,1.428027,...,3225.764300,116.864743,0.252568,3225.890584,3225.902001,-0.278490,2604990.0,3226.50,162670.754717,0.737705
4,1182,2020-07-28 08:30:04.712935671-05:00,3225.75,3226.00,32.0,64.0,9963563,13:30:04.705000000,ESU0,1.428027,...,3226.653391,164.676376,0.256947,3226.781864,3226.809739,-0.553747,2441690.0,3225.75,163684.446043,0.805195


In [6]:
bars_df.columns

Index(['original_index', 'Date-Time', 'Bid Price', 'Ask Price', 'Bid Size',
       'Ask Size', 'Seq. No.', 'Exch Time', '#RIC', 'dailyVolatility', 'Time',
       'Type', 'Spread', 'Mid Quote', 'Smart Price', 'Quote Imbalance',
       'pt_long_ind', 'sl_long_ind', 'pt_short_ind', 'sl_short_ind', 'end_ind',
       'long_label', 'short_label', 'TW Avg Ask Price', 'TW Avg Ask Size',
       'TW Avg Bid Price', 'TW Avg Bid Size', 'TW Avg Spread',
       'TW Avg Mid Quote', 'TW Avg Smart Price', 'TW Avg Quote Imbalance',
       'Order Imbalance', 'Close Price', 'Avg Trade Size',
       'Net SellBuy Count'],
      dtype='object')

In [28]:
df_quotes = trim_df_to_time(df_quotes, "8:30:00.00", "15:00:00.00")

full df shape (560937, 10)
>8:30  df shape (472311, 10)
time-trimmed df shape (457751, 10)


In [29]:
print(df_quotes.shape)

(457751, 10)


In [30]:
df_quotes.dtypes

Date-Time          datetime64[ns, pytz.FixedOffset(-360)]
Bid Price                                         float64
Ask Price                                         float64
Bid Size                                          float64
Ask Size                                          float64
Seq. No.                                            int64
Exch Time                                          object
#RIC                                               object
dailyVolatility                                     int64
Time                                       datetime64[ns]
dtype: object